<a href="https://colab.research.google.com/github/semajyllek/ctmatch/blob/main/bert_fine_tune_ct21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
!nvidia-smi

Wed Dec 15 19:42:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments, AutoModelForSequenceClassification
from drive.MyDrive.trec21_ct_full.trec_ct.scripts.bert_fine_tune import *
from drive.MyDrive.trec21_ct_full.trec_ct.scripts.trec21_vis import *
from sklearn.metrics import f1_score
from collections import defaultdict
from datasets import load_dataset
from numpy.linalg import norm
from numpy import dot
from torch import nn
import numpy as np
import sklearn
import random
import torch


In [ ]:
label_counts, test_rel_dict, all_qrelled_docs = analyze_test_rels('/content/drive/MyDrive/trec21_ct_full/A_Test_Collection_for_Matching_Patient_to_Clinical_Trials/qrels-clinical_trials.txt')

Rel Type Results:
0: 2764
1: 685
2: 421
20141 143
201410 60
201411 42
201412 53
201413 63
201414 73
201415 13
201416 58
201417 50
201418 74
201419 93
20142 62
201420 75
201421 92
201422 40
201423 75
201424 71
201425 53
201426 35
201427 41
201429 71
20143 99
201430 67
20144 33
20145 80
20146 108
20147 153
20148 69
20149 48
20151 77
201510 48
201511 71
201512 72
201513 66
201514 43
201515 91
201516 65
201517 45
201518 118
201519 71
20152 64
201520 47
201521 68
201522 60
201523 38
201524 76
201525 52
201526 38
201527 43
201528 24
201529 20
20153 118
201530 37
20154 86
20155 59
20156 84
20157 103
20158 40
20159 52
Total relled: 3626


In [ ]:
test_docs_path = '/content/drive/MyDrive/ct_data/proc_test_docs.jsonl'
test_docs_ml_path = '/content/drive/MyDrive/ct_data/proc_test_ml_docs.jsonl'
test_topics_path = '/content/drive/MyDrive/ct_data/proc_test_topics.jsonl'
test_topics_ml_path = '/content/drive/MyDrive/ct_data/proc_test_ml_topics.jsonl'

In [ ]:
#truncate_sents(test_docs_path, test_docs_ml_path, max_lens=[254, 255], labels=["include_criteria", "exclude_criteria"])

In [ ]:
docs = load_results(test_docs_ml_path)
docs[:5]

[{'exclude_criteria': 'Age < 1 year or > 21 years Patients who have known allergy to mebendazole or benzimidazole class drugs. Patients who have previously had a severe side effect, such as agranulocytosis and neutropenia, in conjunction with previous mebendazole or benzimidazole class drug for a parasitic infection . Patients who are taking metronidazole and cannot be safely moved to a different antibiotic greater than 7 days prior to starting mebendazole therapy. Pregnant female patients are not eligible for this study. Pregnancy tests with a negative result must be obtained in all post-menarchal females. Lactating females must agree they will not breastfeed a child while on this study. Males and females of reproductive potential may not participate unless they agree to use an effective contraceptive method and continue to do so for at least 6 months after the completion of therapy. Patients who are unable to take oral medications because of significant vomiting will be Group A Low-g

In [ ]:
#truncate_sents(test_topics_path, test_topics_ml_path, max_lens=[510], labels=["sents"])
topics = load_results(test_topics_ml_path)
topics[:5]

[{'id': '20141',
  'sents': 'A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the back, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes.'},
 {'id': '20142',
  'sents': 'An 8-year-old male presents in March to the ER with fever up to 39 C, dyspnea and cough for 2 days. He has just returned from a 5 day vacation in Colorado. Parents report that prior to the onset of fever and cough, he had loose stools. He denies upper respiratory tract symptoms. On examination he is in r

In [ ]:
id2doc = {doc['id']:doc for doc in docs}
id2topic = {topic['id']:topic for topic in topics}

In [ ]:
topic_key = list(test_rel_dict.keys())[11]
topic_key, get_relled(topic_key, test_rel_dict)

('20142',
 {'ones': {'NCT00237016',
   'NCT01099943',
   'NCT01253980',
   'NCT01269554',
   'NCT02618655'},
  'twos': {'NCT00711399',
   'NCT01017081',
   'NCT01707485',
   'NCT02269761',
   'NCT02380352'},
  'zeros': {'NCT00000932',
   'NCT00086879',
   'NCT00094887',
   'NCT00095316',
   'NCT00096772',
   'NCT00132951',
   'NCT00161772',
   'NCT00161798',
   'NCT00161824',
   'NCT00161863',
   'NCT00161967',
   'NCT00163540',
   'NCT00163618',
   'NCT00311441',
   'NCT00385632',
   'NCT00386035',
   'NCT00432003',
   'NCT00455468',
   'NCT00460486',
   'NCT00498654',
   'NCT00503529',
   'NCT00507871',
   'NCT00513500',
   'NCT00617344',
   'NCT00680836',
   'NCT00694655',
   'NCT00740155',
   'NCT00840801',
   'NCT00851487',
   'NCT00894686',
   'NCT01070732',
   'NCT01075204',
   'NCT01087892',
   'NCT01147445',
   'NCT01177657',
   'NCT01201252',
   'NCT01202201',
   'NCT01241201',
   'NCT01253967',
   'NCT01359020',
   'NCT01521403',
   'NCT01589926',
   'NCT01627327',
   'NCT01

In [ ]:
first_topic = id2topic[list(id2topic.keys())[0]];first_topic['sents']

'A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the back, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes.'

In [ ]:
ct_ml_datafile = "/content/drive/MyDrive/ct_ml_data/ct_data.jsonl"
#create_dataset(id2doc, id2topic, test_rel_dict, writefile=ct_ml_datafile)


In [ ]:
ct_dataset = load_dataset("json", data_files=ct_ml_datafile);ct_dataset

Using custom data configuration default-8df2be64c572bd88
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-8df2be64c572bd88/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['topic_text', 'crit_text', 'label'],
        num_rows: 3870
    })
})

In [ ]:
ct_dataset = train_test_val_split(ct_dataset, splits={"train":0.8, "val":0.1})

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/json/default-8df2be64c572bd88/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-9529187fe246cb53.arrow and /root/.cache/huggingface/datasets/json/default-8df2be64c572bd88/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-d477128800868d75.arrow


In [ ]:
ct_dataset

DatasetDict({
    train: Dataset({
        features: ['topic_text', 'crit_text', 'label'],
        num_rows: 2786
    })
    test: Dataset({
        features: ['topic_text', 'crit_text', 'label'],
        num_rows: 774
    })
    validation: Dataset({
        features: ['topic_text', 'crit_text', 'label'],
        num_rows: 310
    })
})

In [ ]:
ct_dataset_df = ct_dataset["train"].to_pandas()

In [ ]:
ct_dataset_df[(ct_dataset_df.label != 0) & (ct_dataset_df.label != 1)]

,topic_text,crit_text,label
4,A 44-year-old man was recently in an automobil...,Subjects who the investigator believes can and...,2
6,A 44 yo male is brought to the emergency room ...,Adult patients (18-80) who are admitted to the...,2
8,A 21-year-old college student undergoes colono...,Patients who are 18 years or older with a clin...,2
16,A 21-year-old female is evaluated for progress...,Patients older than 18 years old Immune Thromb...,2
35,A physician is called to see a 67-year-old wom...,age 19-90 years having clinical ultrasound eva...,2
...,...,...,...
2747,A 32-year-old woman is admitted to the ER foll...,Road traffic accident victims hospitalized in ...,2
2758,A 32 year old female with no previous medical ...,have not been previously diagnosed with cervic...,2
2768,A 56-year-old female on 20th day post-left mas...,Admitted to Scripps Mercy Trauma Service ≥18 Y...,2
2769,A 32 year old female with no previous medical ...,Referred to follow up due to one or more abnor...,2


In [ ]:
ct_dataset_df.topic_text[1]

'A 26-year-old obese woman with a history of bipolar disorder complains that her recent struggles with her weight and eating have caused her to feel depressed. She states that she has recently had difficulty sleeping and feels excessively anxious and agitated. She also states that she has had thoughts of suicide. She often finds herself fidgety and unable to sit still for extended periods of time. Her family tells her that she is increasingly irritable. Her current medications include lithium carbonate and zolpidem.'

In [ ]:
topics[0]

{'id': '20141',
 'sents': 'A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the back, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes.'}

In [ ]:
get_relled('20141', test_rel_dict)['twos']

{'NCT00005127',
 'NCT00005485',
 'NCT00005757',
 'NCT00143195',
 'NCT00175279',
 'NCT00373828',
 'NCT00683813',
 'NCT00809029',
 'NCT00832442',
 'NCT00952744',
 'NCT01253486',
 'NCT01724567',
 'NCT02381340',
 'NCT02532699'}

In [ ]:
id2doc['NCT00683813']

{'exclude_criteria': 'Previous experience with a cardiac rehabilitation program. Patients with depression, uncontrolled diabetes and other significant co-morbidities that may interfere with effective IHD management. Those patients, who in the mind of the attending physician, are unsuitable for participation. Those unable to provide informed consent. Pregnant women. High-risk patients for safety considerations (future studies will high-risk patients).',
 'id': 'NCT00683813',
 'include_criteria': 'Men and women admitted for an IHD event (acute coronary syndrome or revascularization procedure) who are at low or moderate risk.91 Regular Internet access (home, work or other environment). Over 18 years of age. Permission of the attending physician. Able to read, write and understand English without difficulty. No physical limitations to regular activity.'}

In [ ]:
ct_dataset_df.crit_text[34]

'A primary psychiatric diagnosis of Major Depressive Disorder as defined by: a score of 18 or above in the 17-item Hamilton Depression Scale; Male or female ages 18+; Ability to give consent Diagnosis of Breast Cancer Patients may be either English or Spanish speaking [SEP] Lifetime history of psychosis or bipolar disorder Patients meeting diagnostic statistic manual for mental disorder for alcohol or substance use disorders who require acute detoxification. Current suicide risk. Advanced cancer or other condition that limits remaining life expectancy to less than 6 months. Patients who are receiving effective medication for Depression'

In [ ]:
ct_dataset["train"].features

{'crit_text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'topic_text': Value(dtype='string', id=None)}

In [ ]:
from datasets import Features, ClassLabel, Value
features = Features({
    'crit_text': Value(dtype='string', id=None),
    'label': ClassLabel(names=["not_relevant", "partially_relevant", "relevant"]),
    'topic_text': Value(dtype='string', id=None)
})

In [ ]:
ct_dataset["train"] = ct_dataset["train"].map(lambda x: x, batched=True, features=features)
ct_dataset["test"] = ct_dataset["test"].map(lambda x: x, batched=True, features=features)
ct_dataset["validation"] = ct_dataset["validation"].map(lambda x: x, batched=True, features=features)

  0%|          | 0/3 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-8df2be64c572bd88/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-c055a3ed761c944a.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
ct_dataset["train"].features

{'crit_text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=3, names=['not_relevant', 'partially_relevant', 'relevant'], names_file=None, id=None),
 'topic_text': Value(dtype='string', id=None)}

In [ ]:
features["label"].int2str(2)

'relevant'

In [ ]:
id2label = {idx:features["label"].int2str(idx) for idx in range(3)};id2label

{0: 'not_relevant', 1: 'partially_relevant', 2: 'relevant'}

In [ ]:
label2id = {v:k for k, v in id2label.items()};label2id

{'not_relevant': 0, 'partially_relevant': 1, 'relevant': 2}

In [ ]:
ct_dataset_df["label"].value_counts(normalize=True).sort_index() 

0    0.713209
1    0.180905
2    0.105887
Name: label, dtype: float64

In [ ]:

checkpoint = "allenai/scibert_scivocab_uncased"
scibert_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
scibert_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [ ]:
#checkpoint= "allenai/scibert_scivocab_uncased"
#scibert_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#scibert_model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli", num_labels=3, id2label=id2label, label2id=label2id)

In [ ]:
def tokenize_text(examples):
  return scibert_tokenizer(examples["topic_text"], examples["crit_text"], truncation=True, padding=True, max_length=512)

In [ ]:
ct_dataset = ct_dataset.map(tokenize_text, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-8df2be64c572bd88/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-e315b14b785dd254.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
label_weights = (1 - (ct_dataset_df["label"].value_counts().sort_index() / len(ct_dataset_df))).values
label_weights = torch.from_numpy(label_weights).float().to("cuda")
label_weights

tensor([0.2868, 0.8191, 0.8941], device='cuda:0')

In [ ]:
ct_dataset = ct_dataset.rename_column("label", "labels")
ct_dataset = ct_dataset.rename_column("topic_text", "sentence1")
ct_dataset = ct_dataset.rename_column("crit_text", "sentence2")
ct_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'sentence2', 'input_ids', 'labels', 'token_type_ids', 'sentence1'],
        num_rows: 2786
    })
    test: Dataset({
        features: ['attention_mask', 'sentence2', 'input_ids', 'labels', 'token_type_ids', 'sentence1'],
        num_rows: 774
    })
    validation: Dataset({
        features: ['attention_mask', 'sentence2', 'input_ids', 'labels', 'token_type_ids', 'sentence1'],
        num_rows: 310
    })
})

In [ ]:

class WeightedLossTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    outputs = model(**inputs)
    logits = outputs.get("logits")
    labels = inputs.get("labels")
    loss_func = nn.CrossEntropyLoss(weight=label_weights)
    loss = loss_func(logits, labels)
    return (loss, outputs) if return_outputs else loss




def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  return {"f1":f1}

In [ ]:
batch_size=8
logging_steps=len(ct_dataset["train"]) // batch_size
output_dir = "/content/drive/MyDrive/scibert_finetuned_ct"
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_steps=logging_steps
)

In [ ]:
trainer = WeightedLossTrainer(
    model=scibert_model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=ct_dataset["train"],
    eval_dataset=ct_dataset["validation"],
    tokenizer=scibert_tokenizer
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2.
***** Running training *****
  Num examples = 2786
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1745


Epoch,Training Loss,Validation Loss,F1
1,0.974200,0.850040,0.727109
2,0.788900,0.769780,0.751413
3,0.591900,0.843136,0.766166
4,0.391100,1.043465,0.783333
5,0.250000,1.111378,0.778273


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2.
***** Running Evaluation *****
  Num examples = 310
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/scibert_finetuned_ct/checkpoint-500
Configuration saved in /content/drive/MyDrive/scibert_finetuned_ct/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/scibert_finetuned_ct/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/scibert_finetuned_ct/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/scibert_finetuned_ct/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2.
***** Running Evaluation *****
  Num examples = 310
  Batch size = 8
Saving model checkpo

TrainOutput(global_step=1745, training_loss=0.5980545894464312, metrics={'train_runtime': 498.7527, 'train_samples_per_second': 27.93, 'train_steps_per_second': 3.499, 'total_flos': 3665169908951040.0, 'train_loss': 0.5980545894464312, 'epoch': 5.0})

In [ ]:
predictions = trainer.predict(ct_dataset["test"])

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2.
***** Running Prediction *****
  Num examples = 774
  Batch size = 8


In [ ]:
predictions.metrics

{'test_f1': 0.7071605915070596,
 'test_loss': 1.5876928567886353,
 'test_runtime': 8.4851,
 'test_samples_per_second': 91.219,
 'test_steps_per_second': 11.432}

In [ ]:
ct_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'sentence2', 'input_ids', 'labels', 'token_type_ids', 'sentence1'],
        num_rows: 2786
    })
    test: Dataset({
        features: ['attention_mask', 'sentence2', 'input_ids', 'labels', 'token_type_ids', 'sentence1'],
        num_rows: 774
    })
    validation: Dataset({
        features: ['attention_mask', 'sentence2', 'input_ids', 'labels', 'token_type_ids', 'sentence1'],
        num_rows: 310
    })
})

In [ ]:
def get_embedding_similarity(topic, document, model, tokenizer):
    topic_input = tokenizer(topic, return_tensors='pt').to('cuda')
    doc_input = tokenizer(document, return_tensors='pt').to('cuda')
    topic_output = model(**topic_input, output_hidden_states=True)
    doc_output = model(**doc_input, output_hidden_states=True)
    topic_last_hidden = np.squeeze(topic_output.hidden_states[-1].detach().cpu().numpy(), axis=0)
    doc_last_hidden = np.squeeze(doc_output.hidden_states[-1].detach().cpu().numpy(), axis=0)
    topic_emb = np.mean(topic_last_hidden, axis=0)
    doc_emb = np.mean(doc_last_hidden, axis=0)
    return dot(topic_emb, doc_emb)/(norm(topic_emb) * norm(doc_emb))

In [ ]:
test_topic = ct_dataset_df.topic_text[34]
test_doc = ct_dataset_df.crit_text[34]

In [ ]:
test_topic

'A 20 yo female college student with no significant past medical history presents with a chief complaint of fatigue. She reports increased sleep and appetite over the past few months as well as difficulty concentrating on her schoolwork. She no longer enjoys spending time with her friends and feels guilty for not spending more time with her family. Her physical exam and laboratory tests, including hemoglobin, hematocrit and thyroid stimulating hormone, are within normal limits.'

In [ ]:
test_doc

'A primary psychiatric diagnosis of Major Depressive Disorder as defined by: a score of 18 or above in the 17-item Hamilton Depression Scale; Male or female ages 18+; Ability to give consent Diagnosis of Breast Cancer Patients may be either English or Spanish speaking [SEP] Lifetime history of psychosis or bipolar disorder Patients meeting diagnostic statistic manual for mental disorder for alcohol or substance use disorders who require acute detoxification. Current suicide risk. Advanced cancer or other condition that limits remaining life expectancy to less than 6 months. Patients who are receiving effective medication for Depression'

In [ ]:
get_embedding_similarity(test_topic, test_doc, scibert_model, scibert_tokenizer)

0.81593513

In [ ]:
def calc_embedding_scores(topics, docs, model, tokenizer, rel_dict):
  embedding_scores = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
  for topic in topics:
    for doc in docs:
      if doc['id'] in rel_dict[topic['id']]:
        embedding_scores[topic['id']][doc['id']]["include_sim"] = get_embedding_similarity(
                                                      topic['sents'], 
                                                      doc['include_criteria'],
                                                      model,
                                                      tokenizer
                                                   )
        embedding_scores[topic['id']][doc['id']]["exclude_sim"] = get_embedding_similarity(
                                                      topic['sents'], 
                                                      doc['exclude_criteria'],
                                                      model,
                                                      tokenizer
                                                   )
  return embedding_scores



In [ ]:
emb_scores = calc_embedding_scores(
    topics, 
    docs, 
    scibert_model,
    scibert_tokenizer,
    test_rel_dict
)

In [ ]:
list(emb_scores.items())[0]

('20141',
 defaultdict(<function __main__.calc_embedding_scores.<locals>.<lambda>.<locals>.<lambda>>,
             {'NCT00000408': defaultdict(float,
                          {'exclude_sim': 0.7815711,
                           'include_sim': 0.7351871}),
              'NCT00000492': defaultdict(float,
                          {'exclude_sim': 0.272154,
                           'include_sim': 0.67688954}),
              'NCT00000501': defaultdict(float,
                          {'exclude_sim': 0.63588315,
                           'include_sim': 0.82279164}),
              'NCT00001853': defaultdict(float,
                          {'exclude_sim': 0.8444592,
                           'include_sim': 0.79508036}),
              'NCT00004727': defaultdict(float,
                          {'exclude_sim': 0.81957793,
                           'include_sim': 0.85952723}),
              'NCT00005127': defaultdict(float,
                          {'exclude_sim': 0.272154, 'include_sim'

In [ ]:
def check_embeddings(trials=5, include_or_exclude="include_sim"):
  for _ in range(trials):
    rnum = int(random.random() * len(emb_scores))
    r_topic, r_dict = list(emb_scores.items())[rnum]
    rells = get_relled(r_topic, test_rel_dict)
    if (len(rells['twos']) == 0) or (len(rells['ones']) == 0) or (len(rells['zeros']) == 0):
      continue

    r_2doc = random.choice(list(rells['twos']))
    print(f"\nrandom relevance-scored doc: {r_2doc}")
    rel_score = emb_scores[r_topic][r_2doc][include_or_exclude]
    print(f"similar (relevant) score: {rel_score}")
    r_1doc = random.choice(list(rells['ones']))
    rel_score = emb_scores[r_topic][r_1doc][include_or_exclude]
    print(f"dissimilar (partial relevance) score: {rel_score}")
    r_0doc = random.choice(list(rells['zeros']))
    rel_score = emb_scores[r_topic][r_0doc][include_or_exclude]
    print(f"dissimilar (no relevance) score: {rel_score}")




In [ ]:
check_embeddings()


random relevance-scored doc: NCT02380352
similar (relevant) score: 0.8363274335861206
dissimilar (partial relevance) score: 0.7072935104370117
dissimilar (no relevance) score: 0.8627672791481018

random relevance-scored doc: NCT01014819
similar (relevant) score: 0.785274088382721
dissimilar (partial relevance) score: 0.7417327761650085
dissimilar (no relevance) score: 0.6528908610343933

random relevance-scored doc: NCT00711399
similar (relevant) score: 0.5971280336380005
dissimilar (partial relevance) score: 0.7072935104370117
dissimilar (no relevance) score: 0.8627672791481018

random relevance-scored doc: NCT01182597
similar (relevant) score: 0.5885850787162781
dissimilar (partial relevance) score: 0.7228937745094299
dissimilar (no relevance) score: 0.6984370350837708


In [ ]:
def eval_embeddings(score_dict, rel_dict, max_rank=1000, inc_or_exc="include_sim"):
  MRR1s, MRR2s, MRRs = [], [], []
  recall_1s, recall_2s = [], []
  recall_maxrank_1s, recall_maxrank_2s = [], []
  MRR10s, MRR100s = [], []
  MRR10_1s, MRR100_1s, MRR10_2s, MRR100_2s = [], [], [], []
  recall_100_1s, recall_100_2s, recall_1000_1s, recall_1000_2s = [], [], [], []

  for topic_id in score_dict:
    MRR1, MRR2, MRR = 0., 0, 0.
    recall_1, recall_2 = 0., 0.
    MRR10_1, MRR10_2, MRR100_1, MRR100_2 = 0., 0., 0., 0.
    MRR10, MRR100 = 0, 0
    recall_100_1, recall_100_2 = 0., 0.
    recall_1000_1, recall_1000_2 = 0., 0.
    recall_maxrank_1, recall_maxrank_2 = 0., 0.
    rells = get_relled(topic_id, rel_dict)
    total_2s = len(rells['twos'])
    total_1s = len(rells['ones'])

   
    if len(score_dict[topic_id]) == 0:
      continue

    if inc_or_exc == "combined":
      mini_score_dict = score_dict[topic_id]
    else:
      mini_score_dict = {docid:d[inc_or_exc] for docid, d in score_dict[topic_id].items()}   #{docid:{inc_score, docid:score,... }
      
          
    if inc_or_exc in {"include_sim", "combined"}:
      reverse = True
    else:
      reverse = False

    for i, doc_id in enumerate(sorted(mini_score_dict, key=mini_score_dict.get, reverse=reverse)):
     
      if doc_id not in rel_dict[topic_id]:
        continue

      if rel_dict[topic_id][doc_id] == 2:
        
        if MRR2 == 0:
          MRR2 = 1 / (1 + i)

        if i <= max_rank:
          recall_maxrank_2 += 1

        if (i <= 10) and (MRR10_2 == 0):
          MRR10_2 = 1 / (i + 1)
          
        if (i <= 100) and (MRR100_2 == 0):
          MRR100_2 = 1 / (1 + i)


        if i <  total_2s:
          recall_2 += 1


        if i < 100:
          recall_100_2 += 1

        if i < 1000:
          recall_1000_2 += 1

      if rel_dict[topic_id][doc_id] == 1:
        if MRR1 == 0:
          MRR1 = 1 / (1 + i)


        if (i <= 10) and (MRR10_1 == 0):
          MRR10_1 = 1 / (i + 1)
          
        if (i <= 100) and (MRR100_1 == 0):
          MRR100_1 = 1 / (1 + i)

        
        if i < 100:
          recall_100_1 += 1

        if i < 1000:
          recall_1000_1 += 1



        if i < max_rank:
          recall_maxrank_1 += 1
        if i <= total_1s + total_2s:
          recall_1 += 1

      MRR1s.append(MRR1)
      MRR2s.append(MRR2)
      MRRs.append(max(MRR1, MRR2))
      r1 =  (recall_1 / total_1s) if (total_1s > 0) else 1
      r2 =  (recall_2 / total_2s) if (total_2s > 0) else 1
      recall_1s.append(r1)
      recall_2s.append(r2)
      rmax1 = (recall_maxrank_1 / total_1s) if (total_1s > 0) else 1
      rmax2 = (recall_maxrank_2 / total_2s) if (total_2s > 0) else 1
      recall_maxrank_1s.append(rmax1)
      recall_maxrank_2s.append(rmax2)

      MRR10s.append(max(MRR10_1, MRR10_2))
      MRR100s.append(max(MRR100_1, MRR100_2))
      MRR10_1s.append(MRR10_1)
      MRR100_1s.append(MRR100_1)
      MRR10_2s.append(MRR10_2)
      MRR100_2s.append(MRR100_2)

      rr100_1 = 1 if (total_1s == 0) else (recall_100_1 / total_1s)
      rr100_2 = 1 if (total_2s == 0) else (recall_100_2 / total_2s)
      rr1000_1 = 1 if (total_1s == 0) else (recall_1000_1 / total_1s)
      rr1000_2 = 1 if (total_2s == 0) else (recall_1000_2 / total_2s)
      recall_100_1s.append(rr100_1)
      recall_1000_1s.append(rr1000_1)
      recall_100_2s.append(rr100_2)
      recall_1000_2s.append(rr1000_2)

  mean_MRR10 = 0 if (len(MRR10s) == 0) else (sum(MRR10s) / len(MRR10s))
  mean_MRR100 = 0 if (len(MRR100s) == 0) else (sum(MRR100s) / len(MRR100s))
  mean_recall_100_1 = 1 if (len(recall_100_1s) == 0) else (sum(recall_100_1s) / len(recall_100_1s))
  mean_recall_1000_1 = 1 if (len(recall_1000_1s) == 0) else (sum(recall_1000_1s) / len(recall_1000_1s))
  mean_recall_100_2 = 1 if (len(recall_100_2s) == 0) else (sum(recall_100_2s) / len(recall_100_2s))
  mean_recall_1000_2 = 1 if (len(recall_1000_2s) == 0) else (sum(recall_1000_2s) / len(recall_1000_2s))


  print(f"Mean MRR1: {sum(MRR1s) / len(MRR1s)}")
  print(f"Mean MRR2: {sum(MRR2s) / len(MRR2s)}")
  print(f"Mean MRR: {sum(MRRs) / len(MRRs)}")
  print(f"Mean recall 1s: {sum(recall_1s) / len(recall_1s)}")
  print(f"Mean recall 2s: {sum(recall_2s) / len(recall_2s)}")
  print(f"recall\@1000 1s: {sum(recall_maxrank_1s) / len(recall_maxrank_1s)}")
  print(f"recall\@1000 2s: {sum(recall_maxrank_2s) / len(recall_maxrank_2s)}")  

  print(f"MRR@10: {mean_MRR10}, MRR@100: {mean_MRR100}")
  print(f"recall_1@100: {mean_recall_100_1}, recall_2@100: {mean_recall_100_2}, recall_1@1000: {mean_recall_1000_1}, recall_2@1000: {mean_recall_1000_2}")



def compute_mixed_scores(score_dict, W=0.3, embed_scores=False):
  new_score_dict = defaultdict(lambda : defaultdict(float))
  if embed_scores:
    inc_field = "include_sim"
    exc_field = "exclude_sim"
  else:
    inc_field = "include_score"
    exc_field = "exclude_score"

  for topic_id in score_dict:
    for doc_id in score_dict[topic_id]:

      inc_score = score_dict[topic_id][doc_id][inc_field]
      exc_score = score_dict[topic_id][doc_id][exc_field]
      new_score_dict[topic_id][doc_id] = inc_score - W * exc_score
  return new_score_dict



In [ ]:
eval_embeddings(emb_scores, test_rel_dict, inc_or_exc="include_sim")

Mean MRR1: 0.40928436754651953
Mean MRR2: 0.3806778937110325
Mean MRR: 0.6295915832207045
Mean recall 1s: 0.3197190082116082
Mean recall 2s: 0.18630975617286313
recall\@1000 1s: 0.547854203511416
recall\@1000 2s: 0.5166933149759483
MRR@10: 0.6267382015056329, MRR@100: 0.6295915832207045
recall_1@100: 0.5418205407214062, recall_2@100: 0.5121688265741539, recall_1@1000: 0.547854203511416, recall_2@1000: 0.5166933149759483


In [ ]:
eval_embeddings(emb_scores, test_rel_dict, inc_or_exc="exclude_sim")

Mean MRR1: 0.421966634077602
Mean MRR2: 0.37399628303282517
Mean MRR: 0.6172588768496162
Mean recall 1s: 0.2765018379483627
Mean recall 2s: 0.23289606435452237
recall\@1000 1s: 0.5314614728227929
recall\@1000 2s: 0.6022656760135258
MRR@10: 0.6157334604815178, MRR@100: 0.6172588768496162
recall_1@100: 0.5254088026978846, recall_2@100: 0.5970566452391383, recall_1@1000: 0.5314614728227929, recall_2@1000: 0.6022656760135258


In [ ]:
combined_emb_scores = compute_mixed_scores(emb_scores, W=1, embed_scores=True)

In [ ]:
eval_embeddings(combined_emb_scores, test_rel_dict, inc_or_exc="combined")

Mean MRR1: 0.470387423265137
Mean MRR2: 0.36767973905559154
Mean MRR: 0.6468294160759669
Mean recall 1s: 0.31553426605470636
Mean recall 2s: 0.20296634361859323
recall\@1000 1s: 0.5704371196870445
recall\@1000 2s: 0.6086307165159678
MRR@10: 0.6455557606332826, MRR@100: 0.6468294160759669
recall_1@100: 0.564804469473151, recall_2@100: 0.6047005221447148, recall_1@1000: 0.5704371196870445, recall_2@1000: 0.6086307165159678


In [ ]:
import json
combined_score_path = '/content/drive/MyDrive/ct_data/combined_scores'

with open(combined_score_path, 'r') as rf:
  sparse_scores = json.load(rf)

In [ ]:
def combine_scores(scores1, scores2, W=1):
  new_scores = defaultdict(lambda:defaultdict(float))
  for topic_id in scores1:
    for doc_id in scores1[topic_id]:
      new_scores[topic_id][doc_id] = scores1[topic_id][doc_id] + W * scores2[topic_id][doc_id]
  return new_scores

In [ ]:
# normaliuze sparse scores

def get_max_score(scores):
  min_score = 999.0
  max_score = 0.0
  for topic in scores:
    for doc in scores[topic]:
      if scores[topic][doc] > max_score:
        max_score = scores[topic][doc]

      if scores[topic][doc] < min_score:
        min_score = scores[topic][doc]
  return min_score, max_score

def normalize_sparse_scores(sparse_scores):
  min_score, max_score = get_max_score(sparse_scores)

  # first get them all > 0, new min is 0
  new_scores = defaultdict(lambda:defaultdict(float))
  for topic in sparse_scores:
    for doc in sparse_scores[topic]:
      if min_score < 0:
        offset = -1 * min_score
      else:
        offset = 0

      new_scores[topic][doc] = (sparse_scores[topic][doc] + offset) / max_score
  return new_scores
      

  

In [ ]:
new_sparse_scores = normalize_sparse_scores(sparse_scores)

In [ ]:
new_emb_scores = defaultdict(lambda: defaultdict(float))
for topic_id, scores in emb_scores.items():
  for doc_id, scores2 in scores.items(): 
    new_emb_scores[topic_id][doc_id] = sum(scores2.values()) / 2
    

In [ ]:
#list(new_emb_scores.items())[0]

In [ ]:
new_scores = combine_scores(new_emb_scores, new_sparse_scores)

defaultdict(<function __main__.combine_scores.<locals>.<lambda>>,
            {'20141': defaultdict(float, {'NCT00004727': 1.624571447475933})})

In [ ]:
def evaluate_combined(score_dict, rel_dict, max_rank=1000):
  recall_2s, recall_1s = [], []
  recall_maxrank_2s, recall_maxrank_1s =  [], []
  MRR1s, MRR2s, MRRs = [], [], []
  MRR10s, MRR100s = [], []
  MRR10_1s, MRR100_1s, MRR10_2s, MRR100_2s  = [], [], [], []
  recall_100_1s, recall_100_2s, recall_1000_1s, recall_1000_2s = [], [], [], [] 

  for topic_id in score_dict:
    relled = get_relled(topic_id, rel_dict)
    total_2s = len(relled['twos'])
    total_1s = len(relled['ones'])
    recall_2, recall_1 = 0., 0.
    recall_maxrank_2, recall_maxrank_1 = 0., 0.
    MRR1, MRR2 = 0., 0.
    MRR10_1, MRR10_2, MRR100_1, MRR100_2 = 0., 0., 0., 0.
    recall_100_1, recall_100_2, recall_1000_1, recall_1000_2 = 0., 0., 0., 0.
    for i, doc_id in enumerate(sorted(score_dict[topic_id], key=score_dict[topic_id].get, reverse=True)):
      if i > max_rank:
        break

      
      if doc_id in rel_dict[topic_id]:
        actual_rel = rel_dict[topic_id][doc_id]
      else:
        actual_rel = 0

      #print(topic_id, doc_id, score_dict[topic_id][doc_id], actual_rel, i)
      if actual_rel == 2:
        if MRR2 == 0:
          MRR2 = 1 / (i + 1)

        if i <= total_2s:
          if (i <= 10) and (MRR10_2 == 0):
            MRR10_2 = 1 / (i + 1)
          
          if (i <= 100) and (MRR100_2 == 0):
            MRR100_2 = 1 / (1 + i)



          if i < 100:
            recall_100_2 += 1

          if i < 1000:
            recall_1000_2 += 1
          recall_2 += 1

        
        recall_maxrank_2 += 1
      elif actual_rel == 1:
        if (i <= 10) and (MRR10_1 == 0):
            MRR10_1 = 1 / (i + 1)

        if (i <= 10) and (MRR100_1 == 0):
            MRR100_1 = 1 / (i + 1)

        if i < 100:
          recall_100_1 += 1

        if i < 1000:
          recall_1000_1 += 1


        if MRR1 == 0:
          MRR1 = 1 / (i + 1)
        if i <= total_1s + total_2s:
          recall_1 += 1
        recall_maxrank_1 += 1
    MRR = max(MRR1, MRR2)
    r2 = 1 if (total_2s == 0) else (recall_2 / total_2s)
    r1 = 1 if (total_1s == 0) else (recall_1 / total_1s)
    recall_2s.append(r2)
    recall_1s.append(r1)
    rm2 = 1 if (total_2s == 0) else (recall_maxrank_2 / total_2s)
    rm1 = 1 if (total_1s == 0) else (recall_maxrank_1 / total_1s)
    recall_maxrank_2s.append(rm2)
    recall_maxrank_1s.append(rm1)
    MRR1s.append(MRR1)
    MRR2s.append(MRR2)
    MRRs.append(MRR)
    MRR10s.append(max(MRR10_1, MRR10_2))
    MRR100s.append(max(MRR100_1, MRR100_2))
    MRR10_1s.append(MRR10_1)
    MRR100_1s.append(MRR100_1)
    MRR10_2s.append(MRR10_2)
    MRR100_2s.append(MRR100_2)

    rr100_1 = 1 if (total_1s == 0) else (recall_100_1 / total_1s)
    rr100_2 = 1 if (total_2s == 0) else (recall_100_2 / total_2s)
    rr1000_1 = 1 if (total_1s == 0) else (recall_1000_1 / total_1s)
    rr1000_2 = 1 if (total_2s == 0) else (recall_1000_2 / total_2s)
    recall_100_1s.append(rr100_1)
    recall_1000_1s.append(rr1000_1)
    recall_100_2s.append(rr100_2)
    recall_1000_2s.append(rr1000_2)

  mean_MRR10 = 0 if (len(MRR10s) == 0) else (sum(MRR10s) / len(MRR10s))
  mean_MRR100 = 0 if (len(MRR100s) == 0) else (sum(MRR100s) / len(MRR100s))
  mean_recall_100_1 = 1 if (len(recall_100_1s) == 0) else (sum(recall_100_1s) / len(recall_100_1s))
  mean_recall_1000_1 = 1 if (len(recall_1000_1s) == 0) else (sum(recall_1000_1s) / len(recall_1000_1s))
  mean_recall_100_2 = 1 if (len(recall_100_2s) == 0) else (sum(recall_100_2s) / len(recall_100_2s))
  mean_recall_1000_2 = 1 if (len(recall_1000_2s) == 0) else (sum(recall_1000_2s) / len(recall_1000_2s))
  mean_MRR = 0 if (len(MRRs) == 0) else (sum(MRRs) / len(MRRs))
  mean_MRR1 = 0 if (len(MRR1s) == 0) else (sum(MRR1s) / len(MRR1s))
  mean_MRR2 = 0 if (len(MRR2s) == 0) else (sum(MRR2s) / len(MRR2s))
  mean_recall_2s = 1 if (len(recall_2s) == 0) else (sum(recall_2s) / len(recall_2s))
  mean_recall_1s = 1 if (len(recall_1s) == 0) else (sum(recall_1s) / len(recall_1s))
  mean_recall_maxrank_2s = 1 if (len(recall_maxrank_2s) == 0) else (sum(recall_maxrank_2s) / len(recall_maxrank_2s))
  mean_recall_maxrank_1s = 1 if (len(recall_maxrank_1s) == 0) else (sum(recall_maxrank_1s) / len(recall_maxrank_1s))

  print(f"mean MRR: {mean_MRR}, mean MRR2: {mean_MRR2}, mean MRR1: {mean_MRR1}")
  print(f"mean recall_2s: {mean_recall_2s}, mean recall_1s: {mean_recall_1s}")
  print(f"mean recall\@{max_rank}_2s: {mean_recall_maxrank_2s}, mean recall\@{max_rank}_1s: {mean_recall_maxrank_1s}")
  print(f"MRR@10: {mean_MRR10}, MRR@100: {mean_MRR100}")
  print(f"recall_1@100: {mean_recall_100_1}, recall_2@100: {mean_recall_100_2}, recall_1@1000: {mean_recall_1000_1}, recall_2@1000: {mean_recall_1000_2}")
  return


In [ ]:
evaluate_combined(new_scores, rel_dict=test_rel_dict)

mean MRR: 0.5651466236211998, mean MRR2: 0.31843782391994896, mean MRR1: 0.3872345262438956
mean recall_2s: 0.25815521971296024, mean recall_1s: 0.4137773347681994
mean recall\@1000_2s: 1.0, mean recall\@1000_1s: 1.0
MRR@10: 0.5564843348741653, MRR@100: 0.5564843348741653
recall_1@100: 0.9865995125217457, recall_2@100: 0.25815521971296024, recall_1@1000: 1.0, recall_2@1000: 0.25815521971296024
